
# Analysis Notebook

This notebook can be used to produce graphs for the data produced. It defaults to using all results, but that can be specified. 
*** 

## 0. Setup <a class="anchor" id="0"></a>
This section imports all files and sets up the notebook.

In [ ]:
# Import all pacakges
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Choose which results to use
results_to_use = []  # If this is empty, all files in the folder will be used
WARMUP_EPOCHS = 0 # Number of epochs to ignore.
sim_colours = ['blue', 'green', 'red', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']


all_results = os.listdir('results/')

if results_to_use == [] or results_to_use == [""]:
    results_to_use = all_results
# remove DS_Store from the list
if '.DS_Store' in results_to_use:
    results_to_use.remove('.DS_Store')

# order them alphabetically
results_to_use.sort()
print("Using results: " + str(results_to_use))


In [ ]:
# Create a dictionary. Each key is a set of results. 

all_sim_results = {}

for sim in results_to_use:
    all_sim_results[sim] = {}
    all_sim_results[sim]['configuration'] = pd.read_csv(
        'results/' + sim + '/configuration.txt', header=None, delimiter=":")
    types_of_results = []
    # Get all the filenames in the folder, excluding the extension
    for file in os.listdir('results/' + sim + '/exported_data/'):
        if file.endswith(".npy"):
            types_of_results.append(file[:-4])
    # Create a dictionary for each type of result
    for result in types_of_results:
        all_sim_results[sim][result] = np.load(
            'results/' + sim + '/exported_data/' + result + '.npy')

In [ ]:
# Show all the all_sim_results in the notebook
for set_of_results in results_to_use:
    print("Results: " + set_of_results)
    print(all_sim_results[set_of_results]['configuration'])

## 1. Average Trip Satisfaction  <a class="anchor" id="1"></a>

In [ ]:
plt.clf()
plt.rcParams['figure.titlesize'] = 50  # Title font size
plt.rcParams['figure.labelsize'] = 50  # Axes labels font size
plt.rcParams['axes.titlesize'] = 50  # Title font size
plt.rcParams['axes.labelsize'] = 40  # Axes labels font size
plt.rcParams['xtick.labelsize'] = 30  # X-axis tick labels font size
plt.rcParams['ytick.labelsize'] = 30  # Y-axis tick labels font size
plt.rcParams['figure.figsize'] = (40, 20)  # Figure size
plt.rcParams['lines.markersize'] = 1  # Figure markersize
plt.rcParams['legend.markerscale'] = 15  # Legend markersize
plt.rcParams['legend.fontsize'] = 20  # Legend font size
plt.rcParams['figure.dpi'] = 200  # Legend font size
average_satisfaction_score_results = []
std_satisfaction_score_results = []

for sim in results_to_use:
    average_satisfaction_score_results.append(
        all_sim_results[sim]['average_satisfaction_score'])

for sim in results_to_use:
    std_satisfaction_score_results.append(
        all_sim_results[sim]['std_satisfaction_score'])

for i, history in enumerate(average_satisfaction_score_results):
    average_satisfaction_score_results[i] = history[WARMUP_EPOCHS:]
    std_satisfaction_score_results[i] = std_satisfaction_score_results[i][WARMUP_EPOCHS:]
for i, history in enumerate(average_satisfaction_score_results):
    plt.plot(list(range(len(history))), history,
             'o', ls='none', label=results_to_use[i], color=sim_colours[i])

for i, history in enumerate(std_satisfaction_score_results):
    plt.fill_between(list(range(len(average_satisfaction_score_results[i]))), average_satisfaction_score_results[
                     i] - history, average_satisfaction_score_results[i] + history, alpha=0.2, color=sim_colours[i], interpolate=True)
plt.xlabel('Epoch')
plt.ylabel('Average Trip Satisfaction Score\n')
plt.title('History of Average Trip Satisfaction Score\n')
plt.legend()
plt.show()


## 2. Average Auction Reward per Intersection

In [ ]:
def plot_single_reward_history(average_reward_per_intersection, label, colour, axs):
    for i in range(average_reward_per_intersection.shape[0]):
        for j in range(average_reward_per_intersection.shape[0]):
            axs[i, j].plot(
                average_reward_per_intersection[i, j], 'o', color=colour, label=label)
            axs[i, j].set_title('[' + str(i) + ',' + str(j) + ']')

def plot_std_fill(avg_values, std_fill, colour, axs):
    for i in range(std_fill.shape[0]):
        for j in range(std_fill.shape[0]):
            axs[i, j].fill_between(list(range(len(avg_values[i, j]))), avg_values[i, j] - std_fill[i, j], avg_values[i, j] + std_fill[i, j], alpha=0.2, color=colour, interpolate=True)

In [ ]:
plt.clf()
plt.rcParams['figure.titlesize'] = 50  # Title font size
plt.rcParams['figure.labelsize'] = 45  # Axes labels font size
plt.rcParams['axes.titlesize'] = 30  # Title font size
plt.rcParams['axes.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = 30  # X-axis tick labels font size
plt.rcParams['ytick.labelsize'] = 30  # Y-axis tick labels font size
plt.rcParams['figure.figsize'] = (30, 20)  # Figure size
plt.rcParams['lines.markersize'] = 4  # Figure markersize
plt.rcParams['legend.markerscale'] = 3  # Legend markersize
plt.rcParams['legend.fontsize'] = 25  # Legend font size
plt.rcParams['figure.dpi'] = 200  # Legend font size

average_reward_per_intersection_history_results = []
std_reward_per_intersection_history_results = []
for sim in results_to_use:
    average_reward_per_intersection_history_results.append(
        all_sim_results[sim]['average_reward_per_intersection_history'])

fig, axs = plt.subplots(
    average_reward_per_intersection_history_results[0].shape[0],
    average_reward_per_intersection_history_results[0].shape[1], sharex=True, sharey=True)

for i, history in enumerate(average_reward_per_intersection_history_results):
    plot_single_reward_history(history, results_to_use[i], sim_colours[i], axs)
    
for i, history in enumerate(std_reward_per_intersection_history_results):    
    plot_std_fill(average_reward_per_intersection_history_results[i], std_reward_per_intersection_history_results[i], sim_colours[i], axs)

fig.supxlabel('Epoch\n')
fig.supylabel('\nAverage Auction Reward')
fig.suptitle(
    '\n History of Average Auction Reward per Intersection')

plt.subplots_adjust(right=0.9)
lgnd = plt.legend(loc='upper left', bbox_to_anchor=(1, 1.5))
plt.show()


## 3. Average Auction Reward Central Intersection

In [ ]:
import statsmodels.api as sm

plt.clf()
plt.rcParams['figure.titlesize'] = 50  # Title font size
plt.rcParams['figure.labelsize'] = 45  # Axes labels font size
plt.rcParams['axes.titlesize'] = 30  # Title font size
plt.rcParams['axes.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = 30  # X-axis tick labels font size
plt.rcParams['ytick.labelsize'] = 30  # Y-axis tick labels font size
plt.rcParams['figure.figsize'] = (30, 20)  # Figure size
plt.rcParams['lines.markersize'] = 4  # Figure markersize
plt.rcParams['legend.markerscale'] = 3  # Legend markersize
plt.rcParams['legend.fontsize'] = 25  # Legend font size
plt.rcParams['figure.dpi'] = 200  # Legend font size

average_reward_per_intersection_history_results = []
std_reward_per_intersection_history_results = []
for sim in results_to_use:
    average_reward_per_intersection_history_results.append(
        all_sim_results[sim]['average_reward_per_intersection_history'][1,1])


for i, history in enumerate(average_reward_per_intersection_history_results):
    model = sm.OLS(history, sm.add_constant(np.arange(len(history)))).fit()
    print("\subsubsection{", results_to_use[i], "}")
    print(model.summary().tables[1].as_latex_tabular())
    
    plt.plot(list(range(len(history))), history,
             'o', ls='none', label=results_to_use[i], color=sim_colours[i])
plt.xlabel('Epoch')
plt.ylabel('Average Auction Reward\n')
plt.title('History of Average Auction Reward\n')
plt.legend()
plt.show()



## 4. Average Throughput per Intersection <a id="3"></a>

In [ ]:
def plot_throughput_history(throughput_history_per_intersection, label, colour, axs):
    for i in range(throughput_history_per_intersection.shape[0]):
        for j in range(throughput_history_per_intersection.shape[0]):
            axs[i, j].plot(
                throughput_history_per_intersection[i, j], 'o', color=colour, label=label)
            axs[i, j].set_title('[' + str(i) + ',' + str(j) + ']')

In [ ]:
plt.clf()
plt.rcParams['figure.titlesize'] = 50  # Title font size
plt.rcParams['figure.labelsize'] = 45  # Axes labels font size
plt.rcParams['axes.titlesize'] = 30  # Title font size
plt.rcParams['axes.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = 30  # X-axis tick labels font size
plt.rcParams['ytick.labelsize'] = 30  # Y-axis tick labels font size
plt.rcParams['figure.figsize'] = (30, 20)  # Figure size
plt.rcParams['lines.markersize'] = 4  # Figure markersize
plt.rcParams['legend.markerscale'] = 3  # Legend markersize
plt.rcParams['legend.fontsize'] = 25  # Legend font size
plt.rcParams['figure.dpi'] = 200  # Legend font size

average_throughput_per_intersection_history_results = []
std_throughput_per_intersection_history_results = []

for sim in results_to_use:
    average_throughput_per_intersection_history_results.append(
        all_sim_results[sim]['average_throughput_per_intersection'])

for sim in results_to_use:
    std_throughput_per_intersection_history_results.append(
        all_sim_results[sim]['std_throughput_per_intersection'])

fig, axs = plt.subplots(
    average_throughput_per_intersection_history_results[0].shape[0],
    average_throughput_per_intersection_history_results[0].shape[1], sharex=True, sharey=True)

for i, history in enumerate(average_throughput_per_intersection_history_results):
    plot_throughput_history(history, results_to_use[i], sim_colours[i], axs)

for i, history in enumerate(std_throughput_per_intersection_history_results):
    plot_std_fill(average_throughput_per_intersection_history_results[i],
                  std_throughput_per_intersection_history_results[i], sim_colours[i], axs)

fig.supxlabel('Epoch\n')
fig.supylabel('\nAverage Throughput')
fig.suptitle('Average Throughput per Intersection ')
plt.subplots_adjust(right=0.9)
lgnd = plt.legend(loc='upper left', bbox_to_anchor=(1, 1.5))
plt.show()


## 5. Average Average Time Waited per Intersection <a id="4"></a>

In [ ]:

def plot_average_time_waited_history(average_time_waited_history_per_intersection, label, colour, axs):
    for i in range(average_time_waited_history_per_intersection.shape[0]):
        for j in range(average_time_waited_history_per_intersection.shape[0]):
            axs[i, j].plot(
                average_time_waited_history_per_intersection[i, j], 'o', color=colour, label=label)
            axs[i, j].set_title('[' + str(i) + ',' + str(j) + ']')
            

In [ ]:
plt.clf()
plt.rcParams['figure.titlesize'] = 50  # Title font size
plt.rcParams['figure.labelsize'] = 45  # Axes labels font size
plt.rcParams['axes.titlesize'] = 30  # Title font size
plt.rcParams['axes.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = 30  # X-axis tick labels font size
plt.rcParams['ytick.labelsize'] = 30  # Y-axis tick labels font size
plt.rcParams['figure.figsize'] = (30, 20)  # Figure size
plt.rcParams['lines.markersize'] = 1  # Figure markersize
plt.rcParams['legend.markerscale'] = 15  # Legend markersize
plt.rcParams['legend.fontsize'] = 25  # Legend font size
plt.rcParams['figure.dpi'] = 200  # Legend font size

average_average_time_waited_per_intersection_history_results = []
std_average_time_waited_per_intersection_history_results = []

for sim in results_to_use:
    average_average_time_waited_per_intersection_history_results.append(
        all_sim_results[sim]['average_average_time_waited_per_intersection_history'])

for sim in results_to_use:
    std_average_time_waited_per_intersection_history_results.append(
        all_sim_results[sim]['std_average_time_waited_per_intersection_history'])

fig, axs = plt.subplots(
    average_average_time_waited_per_intersection_history_results[0].shape[0],
    average_average_time_waited_per_intersection_history_results[0].shape[1], sharex=True, sharey=True)

for i, history in enumerate(average_average_time_waited_per_intersection_history_results):
    plot_average_time_waited_history(
        history, results_to_use[i], sim_colours[i], axs)

for i, history in enumerate(std_average_time_waited_per_intersection_history_results):
    plot_std_fill(average_average_time_waited_per_intersection_history_results[i],
                  std_average_time_waited_per_intersection_history_results[i], sim_colours[i], axs)

fig.supxlabel('Epoch\n')
fig.supylabel('\nAverage Time Waited')
fig.suptitle('Average Time Waited per Intersection')
plt.subplots_adjust(right=0.9)
lgnd = plt.legend(loc='upper left', bbox_to_anchor=(1, 1.5))
plt.show()


## 6. Average Max Time Waited per Intersection <a id="4"></a>

In [ ]:
def plot_max_time_waited_history(max_time_waited_history_per_intersection, label, colour, axs):
    for i in range(max_time_waited_history_per_intersection.shape[0]):
        for j in range(max_time_waited_history_per_intersection.shape[0]):
            axs[i, j].plot(
                max_time_waited_history_per_intersection[i, j], 'o', color=colour, label=label)
            axs[i, j].set_title('[' + str(i) + ',' + str(j) + ']')
            axs[i, j].set_xlim([0, 1000])

In [ ]:
plt.clf()
plt.rcParams['figure.titlesize'] = 50  # Title font size
plt.rcParams['figure.labelsize'] = 45  # Axes labels font size
plt.rcParams['axes.titlesize'] = 30  # Title font size
plt.rcParams['axes.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = 30  # X-axis tick labels font size
plt.rcParams['ytick.labelsize'] = 30  # Y-axis tick labels font size
plt.rcParams['figure.figsize'] = (30, 20)  # Figure size
plt.rcParams['lines.markersize'] = 1  # Figure markersize
plt.rcParams['legend.markerscale'] = 15  # Legend markersize
plt.rcParams['legend.fontsize'] = 25  # Legend font size
plt.rcParams['figure.dpi'] = 200  # Legend font size

average_max_time_waited_per_intersection_history_results = []
std_max_time_waited_per_intersection_history_results = []

for sim in results_to_use:
    average_max_time_waited_per_intersection_history_results.append(
        all_sim_results[sim]['average_max_time_waited_per_intersection_history'])

for sim in results_to_use:
    std_max_time_waited_per_intersection_history_results.append(
        all_sim_results[sim]['std_max_time_waited_per_intersection_history'])

fig, axs = plt.subplots(
    average_max_time_waited_per_intersection_history_results[0].shape[0],
    average_max_time_waited_per_intersection_history_results[0].shape[1], sharex=True, sharey=True)

for i, history in enumerate(average_max_time_waited_per_intersection_history_results):
    plot_max_time_waited_history(
        history, results_to_use[i], sim_colours[i], axs)

for i, history in enumerate(std_max_time_waited_per_intersection_history_results):
    plot_std_fill(average_max_time_waited_per_intersection_history_results[i],
                  std_max_time_waited_per_intersection_history_results[i], sim_colours[i], axs)

fig.supxlabel('Epoch\n')
fig.supylabel('\nAverage Max Time Waited')
fig.suptitle('Average Max Time Waited per Intersection ')
plt.subplots_adjust(right=0.9)
lgnd = plt.legend(loc='upper left', bbox_to_anchor=(1, 1.5))
plt.show()


## 7. Average Max Time Waited for central Intersection <a id="4"></a>

In [ ]:
plt.clf()
plt.rcParams['figure.titlesize'] = 50  # Title font size
plt.rcParams['figure.labelsize'] = 45  # Axes labels font size
plt.rcParams['axes.titlesize'] = 30  # Title font size
plt.rcParams['axes.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = 30  # X-axis tick labels font size
plt.rcParams['ytick.labelsize'] = 30  # Y-axis tick labels font size
plt.rcParams['figure.figsize'] = (30, 20)  # Figure size
plt.rcParams['lines.markersize'] = 3  # Figure markersize
plt.rcParams['legend.markerscale'] = 5  # Legend markersize
plt.rcParams['legend.fontsize'] = 25  # Legend font size
plt.rcParams['figure.dpi'] = 200  # Legend font size

average_max_time_waited_per_intersection_history_results = []

for sim in results_to_use:
    average_max_time_waited_per_intersection_history_results.append(
        all_sim_results[sim]['average_max_time_waited_per_intersection_history'][1,1])


for i, history in enumerate(average_max_time_waited_per_intersection_history_results):
    model = sm.OLS(history, sm.add_constant(np.arange(len(history)))).fit()
    plt.plot(list(range(len(history))), history,
             'o', ls='none', label=results_to_use[i], color=sim_colours[i])
plt.xlim([0, 2000])
plt.xlabel('Epoch')
plt.ylabel('Average Max Time Waited\n')
plt.title('History of Max Time Waited \n')
plt.legend()
plt.show()

## 8. Average Gini (based on time waited) coefficient per Intersection <a id="5"></a>

In [ ]:
def plot_gini_history(gini_history_per_intersection, label, colour, axs):
    for i in range(gini_history_per_intersection.shape[0]):
        for j in range(gini_history_per_intersection.shape[0]):
            axs[i, j].plot(
                gini_history_per_intersection[i, j], 'o', color=colour, label=label)
            axs[i, j].set_title('[' + str(i) + ',' + str(j) + ']')

In [ ]:
plt.clf()
plt.rcParams['figure.titlesize'] = 50  # Title font size
plt.rcParams['figure.labelsize'] = 45  # Axes labels font size
plt.rcParams['axes.titlesize'] = 30  # Title font size
plt.rcParams['axes.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = 30  # X-axis tick labels font size
plt.rcParams['ytick.labelsize'] = 30  # Y-axis tick labels font size
plt.rcParams['figure.figsize'] = (30, 20)  # Figure size
plt.rcParams['lines.markersize'] = 1  # Figure markersize
plt.rcParams['legend.markerscale'] = 15  # Legend markersize
plt.rcParams['legend.fontsize'] = 25  # Legend font size
plt.rcParams['figure.dpi'] = 200  # Legend font size

average_gini_per_intersection_history_results = []
std_gini_per_intersection_history_results = []

for sim in results_to_use:
    average_gini_per_intersection_history_results.append(
        all_sim_results[sim]['average_gini_time_waited_per_intersection_history'])

for sim in results_to_use:
    std_gini_per_intersection_history_results.append(
        all_sim_results[sim]['std_gini_time_waited_per_intersection_history'])

fig, axs = plt.subplots(
    average_gini_per_intersection_history_results[0].shape[0],
    average_gini_per_intersection_history_results[0].shape[1], sharex=True, sharey=True)

for i, history in enumerate(average_gini_per_intersection_history_results):
    plot_gini_history(history, results_to_use[i], sim_colours[i], axs)

for i, history in enumerate(std_gini_per_intersection_history_results):
    plot_std_fill(average_gini_per_intersection_history_results[i],
                  std_gini_per_intersection_history_results[i], sim_colours[i], axs)

fig.supxlabel('Epoch\n')
fig.supylabel('\nAverage Gini Coefficient')
fig.suptitle('Average Gini per Intersection (Based on Time Waited)')
plt.subplots_adjust(right=0.9)
lgnd = plt.legend(loc='upper left', bbox_to_anchor=(1, 1.5))
plt.show()


## 9. Average Parameter Valuations per Intersection <a id="5"></a>

In [ ]:
def plot_average_valuation_per_parameter_per_intersection(rewards, parameters, counts, label, colour, axs):
    for i in range(rewards.shape[0]):
        for j in range(rewards.shape[0]):
            axs[i, j].scatter(
                parameters[:, 0], rewards[i, j, :], s=counts[i, j, :], marker="o", c=colour, label=label)
            axs[i, j].set_title('[' + str(i) + ',' + str(j) + ']')

In [ ]:
plt.clf()
plt.rcParams['figure.titlesize'] = 50  # Title font size
plt.rcParams['figure.labelsize'] = 45  # Axes labels font size
plt.rcParams['axes.titlesize'] = 30  # Title font size
plt.rcParams['axes.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = 30  # X-axis tick labels font size
plt.rcParams['ytick.labelsize'] = 30  # Y-axis tick labels font size
plt.rcParams['figure.figsize'] = (30, 20)  # Figure size
plt.rcParams['lines.markersize'] = 4  # Figure markersize
plt.rcParams['legend.markerscale'] = 3  # Legend markersize
plt.rcParams['legend.fontsize'] = 25  # Legend font size
plt.rcParams['figure.dpi'] = 200  # Legend font size

average_valuations_per_parameter_per_intersection = []
parameters_used = []
counts_per_parameter = []
for sim in results_to_use:
    average_valuations_per_parameter_per_intersection.append(
        all_sim_results[sim]['average_reward_per_parameter_set_per_intersection_rewards'])
    parameters_used.append(
        all_sim_results[sim]['average_reward_per_parameter_set_per_intersection_parameters'])
    counts_per_parameter.append(
        all_sim_results[sim]['average_reward_per_parameter_set_per_intersection_counts'])


fig, axs = plt.subplots(
    average_valuations_per_parameter_per_intersection[0].shape[0],
    average_valuations_per_parameter_per_intersection[0].shape[1], sharex=True, sharey=True)

for i, reward_set in enumerate(average_valuations_per_parameter_per_intersection):
    plot_average_valuation_per_parameter_per_intersection(
        reward_set, parameters_used[i], counts_per_parameter[i],  results_to_use[i], sim_colours[i], axs)

fig.supxlabel('Delay Boost Value\n')
fig.supylabel('\n Average Expected Reward')
fig.suptitle('Average Expected Reward per Parameter')
plt.subplots_adjust(right=0.9)
lgnd = plt.legend(loc='upper left', bbox_to_anchor=(1, 1.5))
plt.show()


## 10. Bid and Inact Ranks per Intersection <a id="5"></a>

In [ ]:
def plot_bid_inact_ranks(mean_bid_rank_per_intersection, se_bid_rank_per_intersection, mean_inact_rank_per_intersection, se_inact_rank_per_intersection, label, colour, axs, shift):
    rank_labels = ['Bid Rank', 'Time Waited \n Rank']
    for i in range(mean_bid_rank_per_intersection.shape[0]):
        for j in range(mean_bid_rank_per_intersection.shape[0]):
            rank_means = [mean_bid_rank_per_intersection[i, j],
                          mean_inact_rank_per_intersection[i, j]]
            rank_ses = [se_bid_rank_per_intersection[i, j],
                        se_inact_rank_per_intersection[i, j]]
            # Create a barplot of the bid ranks per intersection
            X_axis = np.arange(len(rank_means))
            axs[i, j].set_xticks([-1, 0, 1, 2])
            axs[i, j].set_xticklabels(['Bid Rank', 'Time Waited \n Rank', 'Bid Rank', 'Time Waited \n Rank'])
                
            # axs[i, j].set_xticks(X_axis + shift)
            # axs[i, j].set_xticklabels(rank_labels)
            axs[i, j].bar(X_axis+shift, rank_means,
                          label=label, yerr=rank_ses, color=colour)


In [ ]:
plt.clf()
plt.rcParams['figure.titlesize'] = 45  # Title font size
plt.rcParams['figure.labelsize'] = 35  # Axes labels font size
plt.rcParams['axes.titlesize'] = 30  # Title font size
plt.rcParams['xtick.labelsize'] = 25  # X-axis tick labels font size
plt.rcParams['ytick.labelsize'] = 25  # Y-axis tick labels font size
plt.rcParams['figure.figsize'] = (40, 20)  # Figure size
plt.rcParams['errorbar.capsize'] = 10  # Error bar capsize

shifts = [-1, 1]
mean_bid_rank_per_intersection_results = []
se_bid_rank_per_intersection_results = []
mean_inact_rank_per_intersection_results = []
se_inact_rank_per_intersection_results = []

for sim in results_to_use:
    mean_bid_rank_per_intersection_results.append(
        all_sim_results[sim]['mean_bid_rank_per_intersection'])
    se_bid_rank_per_intersection_results.append(
        all_sim_results[sim]['se_bid_rank_per_intersection'])
    mean_inact_rank_per_intersection_results.append(
        all_sim_results[sim]['mean_inact_rank_per_intersection'])
    se_inact_rank_per_intersection_results.append(
        all_sim_results[sim]['se_inact_rank_per_intersection'])

fig, axs = plt.subplots(
    mean_bid_rank_per_intersection_results[0].shape[0],
    mean_bid_rank_per_intersection_results[0].shape[1], sharex=False, sharey=True)

for i, (mean_b, se_b, mean_i, se_i) in enumerate(zip(mean_bid_rank_per_intersection_results, se_bid_rank_per_intersection_results, mean_inact_rank_per_intersection_results, se_inact_rank_per_intersection_results)):
    plot_bid_inact_ranks(mean_b, se_b, mean_i, se_i,
                         results_to_use[i], sim_colours[i], axs, shifts[i])

fig.supxlabel('Rank Type\n')
fig.supylabel('\nAverage Rank Value')
fig.suptitle(
    '\n Average Bid Rank and Time Waited Rank of Winner per Intersection')
plt.subplots_adjust(right=0.9)
lgnd = plt.legend(loc='upper left', bbox_to_anchor=(1, 1.5))
plt.show()


## 11. Valuation per Aggression <a id="5"></a>

In [ ]:
def plot_average_valuation_per_parameter_per_intersection(rewards, parameters, counts, label, colour, axs):
    for i in range(rewards.shape[0]):
        for j in range(rewards.shape[0]):
            axs[i, j].scatter(
                parameters[:, 0], rewards[i, j, :], s=counts[i, j, :], marker="o", c=colour, label=label)
            axs[i, j].set_title('[' + str(i) + ',' + str(j) + ']')

In [ ]:
plt.clf()
plt.rcParams['figure.titlesize'] = 50  # Title font size
plt.rcParams['figure.labelsize'] = 50  # Axes labels font size
plt.rcParams['axes.titlesize'] = 50  # Title font size
plt.rcParams['axes.labelsize'] = 40  # Axes labels font size
plt.rcParams['xtick.labelsize'] = 30  # X-axis tick labels font size
plt.rcParams['ytick.labelsize'] = 30  # Y-axis tick labels font size
plt.rcParams['figure.figsize'] = (40, 20)  # Figure size
plt.rcParams['lines.markersize'] = 1  # Figure markersize
plt.rcParams['legend.markerscale'] = 0.5  # Legend markersize
plt.rcParams['legend.fontsize'] = 30  # Legend font size
plt.rcParams['figure.dpi'] = 200  # Legend font size

average_valuations_per_parameter = []
parameters_used = []
counts_per_parameter = []
for sim in results_to_use:
    average_valuations_per_parameter.append(
        all_sim_results[sim]['all_sims_adaptive_bidding_parameters_valuations'])
    parameters_used.append(
        all_sim_results[sim]['all_sims_adaptive_bidding_parameters_space'])
    counts_per_parameter.append(
        all_sim_results[sim]['all_sims_adaptive_bidding_parameters_counts'])
    

for i, set in enumerate(average_valuations_per_parameter):
    plt.scatter(parameters_used[i], set,
            s=counts_per_parameter[i]*20, marker="o", label=results_to_use[i], color=sim_colours[i])



plt.title('Average Expected Trip Satisfaction per Bid Aggression\n')
plt.xlabel('\nBid Aggression')
plt.ylabel('Expected Trip Satisfaction\n')
plt.legend()
plt.show()